In [21]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


In [23]:
# se carga el dataset que se va a utilizar para dos dataframes distintos
data = pd.read_csv('data_steam.csv')
data_juegos_steam = pd.read_csv('data_steam.csv')

# eliminando columnas irrelevantes
data_juegos_steam.drop(['id','user_id','recommend','price','playtime_forever','year_posted','sentiment_analysis','developer','release_date'],axis=1,inplace=True)
data.drop(['id','user_id','recommend','price','playtime_forever','year_posted','sentiment_analysis'],axis=1,inplace=True)

# cambiando tipos de datos
data['item_id'] = data['item_id'].astype(str)
data['release_date'] = data['release_date'].astype(str)

# borrando duplicados
data = data.drop_duplicates(subset='item_id', keep='first')
data_juegos_steam = data_juegos_steam.drop_duplicates(subset='item_id', keep='first')

# Combinar los valores de las columnas en una sola columna
data['features'] = data[['title', 'developer', 'release_date']].apply(lambda x: ', '.join(x), axis=1)

data.drop(['title','developer','release_date'],axis=1,inplace=True)

# Exportar data limpia de juegos
data.to_csv('juegos_steam.csv', index=False)
data_juegos_steam.to_csv('juegos_id.csv', index=False)

ML

In [24]:
def recomendacion_juego(item_id):
    # se carga los datasets que se va a utilizar para dos dataframes distintos
    data = pd.read_csv('juegos_steam.csv')
    data_juegos_steam = pd.read_csv('juegos_id.csv')

    # crear una matriz de características de los juegos
    tfidv = TfidfVectorizer(min_df=2, max_df=0.7, token_pattern=r'\b[a-zA-Z0-9]\w+\b')
    data_vector = tfidv.fit_transform(data['features'])

    data_vector_df = pd.DataFrame(data_vector.toarray(), index=data['item_id'], columns = tfidv.get_feature_names_out())

    # calcular la similitud coseno entre los juegos en la matriz de características
    vector_similitud_coseno = cosine_similarity(data_vector_df.values)

    cos_sim_df = pd.DataFrame(vector_similitud_coseno, index=data_vector_df.index, columns=data_vector_df.index)

    juego_simil = cos_sim_df.loc[item_id]

    simil_ordenada = juego_simil.sort_values(ascending=False)
    resultado = simil_ordenada.head(6).reset_index()

    result_df = resultado.merge(data_juegos_steam, on='item_id',how='left')

    # La función devuelve una lista de los 6 juegos más similares al juego dado
    juego_title = data_juegos_steam[data_juegos_steam['item_id'] == item_id]['title'].values[0]

    # mensaje que indica el juego original y los juegos recomendados
    mensaje = f"Si te gustó el juego {item_id} : {juego_title}, también te pueden gustar:"

    result_dict = {
        'mensaje': mensaje,
        'juegos recomendados': result_df['title'][1:6].tolist()
    }

    return result_dict

In [25]:
raul= recomendacion_juego(1250)
raul

{'mensaje': 'Si te gustó el juego 1250 : Killing Floor, también te pueden gustar:',
 'juegos recomendados': ['Killing Floor 2',
  'Red Orchestra: Ostfront 41-45',
  'Caster',
  'Marauder',
  'Fading Hearts']}